In [17]:
import tensorflow as tf

import os
import gc


from tqdm.autonotebook import tqdm

import numpy as np #
import pandas as pd

# from keras import regularizers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.nasnet import NASNetLarge, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.applications.xception import Xception, preprocess_input
import torch

def get_features(model_name, model_preprocessor, input_size, data):

    input_layer = Input(input_size)
    preprocessor = Lambda(model_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    
    #Extract feature.
    feature_maps = feature_extractor.predict(data, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

def extact_features(data):
    inception_features = get_features(InceptionV3, inception_preprocessor, img_size, data)
    xception_features = get_features(Xception, xception_preprocessor, img_size, data)
    nasnet_features = get_features(NASNetLarge, nasnet_preprocessor, img_size, data)
    inc_resnet_features = get_features(InceptionResNetV2, inc_resnet_preprocessor, img_size, data)

    final_features = np.concatenate([inception_features,
                                     xception_features,
                                     nasnet_features,
                                     inc_resnet_features],axis=-1)

    print('Final feature maps shape', final_features.shape)

    #deleting to free up ram memory
    del inception_features 
    del xception_features
    del nasnet_features
    del inc_resnet_features
    gc.collect()


    return final_features

img_size = (331,331,3)

inception_preprocessor = preprocess_input
xception_preprocessor = preprocess_input
inc_resnet_preprocessor = preprocess_input
nasnet_preprocessor = preprocess_input

loaded_model_tf = tf.keras.models.load_model('classification_model.h5')
labels = pd.read_csv('labels.csv')
classes = sorted(list(set(labels['breed'])))

img_g = load_img('dataset/[경기-수원-2022-00328].jpg',target_size = img_size)
img_g = np.expand_dims(img_g, axis=0) # as we trained our model in (row, img_height, img_width, img_rgb) format, np.expand_dims convert the image into this format
# img_g
# #Predict test labels given test data features.
test_features = extact_features(img_g)
predg = loaded_model_tf.predict(test_features)
print(f"Predicted label: {classes[np.argmax(predg[0])]}")
print(f"Probability of prediction): {round(np.max(predg[0])) * 100} %")

tensor=torch.tensor(predg)

val, idx = torch.sort(tensor,  descending=True)
# 값 top2 값 매핑
print(val)

# 값 true인 index 찍어보면 견종 나옴
print((tensor>0.1).squeeze())



1/1 [==============================] - 1s 554ms/step
Feature maps shape:  (1, 2048)
1/1 [==============================] - 0s 353ms/step
Feature maps shape:  (1, 2048)
1/1 [==============================] - 2s 2s/step
Feature maps shape:  (1, 4032)
1/1 [==============================] - 1s 1s/step
Feature maps shape:  (1, 1536)
Final feature maps shape (1, 9664)
Predicted label: toy_poodle
Probability of prediction): 100 %
tensor([[9.9036e-01, 6.8113e-03, 6.2964e-04, 4.7763e-04, 3.0030e-04, 1.6055e-04,
         1.5749e-04, 1.1635e-04, 7.0684e-05, 4.4619e-05, 4.0008e-05, 3.8993e-05,
         3.5659e-05, 3.4508e-05, 3.1929e-05, 2.9077e-05, 2.8828e-05, 2.6643e-05,
         2.6293e-05, 2.5781e-05, 2.2832e-05, 2.2253e-05, 2.2142e-05, 1.8306e-05,
         1.6396e-05, 1.5855e-05, 1.5797e-05, 1.5763e-05, 1.4359e-05, 1.2708e-05,
         1.2250e-05, 1.2195e-05, 1.1920e-05, 1.1881e-05, 1.1780e-05, 1.1773e-05,
         1.1010e-05, 1.0798e-05, 8.5260e-06, 8.0388e-06, 7.9664e-06, 7.9654e-06,
      

In [16]:
(tensor>0.1).squeeze()[110]
#(tensor>0.001).squeeze() 

tensor(False)

In [9]:
classes[110]

'toy_poodle'